## Week 4, Lab 2: Predicting Chronic Kidney Disease in Patients
> Author: Matt Brems

We can sketch out the data science process as follows:
1. Define the problem.
2. Obtain the data.
3. Explore the data.
4. Model the data.
5. Evaluate the model.
6. Answer the problem.

In this lab, we're going to focus on steps exploring data, building models and evaluating the models we build.

There are three links you may find important:
- [A set of chronic kidney disease (CKD) data and other biological factors](./chronic_kidney_disease_full.csv).
- [The CKD data dictionary](./chronic_kidney_disease_header.txt).
- [An article comparing the use of k-nearest neighbors and support vector machines on predicting CKD](./chronic_kidney_disease.pdf).

## Step 1: Define the problem.

Suppose you're working for Mayo Clinic, widely recognized to be the top hospital in the United States. In your work, you've overheard nurses and doctors discuss test results, then arrive at a conclusion as to whether or not someone has developed a particular disease or condition. For example, you might overhear something like:

> **Nurse**: Male 57 year-old patient presents with severe chest pain. FDP _(short for fibrin degradation product)_ was elevated at 13. We did an echo _(echocardiogram)_ and it was inconclusive.

> **Doctor**: What was his interarm BP? _(blood pressure)_

> **Nurse**: Systolic was 140 on the right; 110 on the left.

> **Doctor**: Dammit, it's an aortic dissection! Get to the OR _(operating room)_ now!

> _(intense music playing)_

In this fictitious but [Shonda Rhimes-esque](https://en.wikipedia.org/wiki/Shonda_Rhimes#Grey's_Anatomy,_Private_Practice,_Scandal_and_other_projects_with_ABC) scenario, you might imagine the doctor going through a series of steps like a [flowchart](https://en.wikipedia.org/wiki/Flowchart), or a series of if-this-then-that steps to diagnose a patient. The first steps made the doctor ask what the interarm blood pressure was. Because interarm blood pressure took on the values it took on, the doctor diagnosed the patient with an aortic dissection.

Your goal, as a research biostatistical data scientist at the nation's top hospital, is to develop a medical test that can improve upon our current diagnosis system for [chronic kidney disease (CKD)](https://www.mayoclinic.org/diseases-conditions/chronic-kidney-disease/symptoms-causes/syc-20354521).

**Real-world problem**: Develop a medical diagnosis test that is better than our current diagnosis system for CKD.

**Data science problem**: Develop a medical diagnosis test that reduces both the number of false positives and the number of false negatives.

---

## Step 2: Obtain the data.

### 1. Read in the data.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.metrics import confusion_matrix, plot_confusion_matrix, recall_score, precision_score, accuracy_score

In [2]:
df = pd.read_csv('chronic_kidney_disease_full.csv')

### 2. Check out the data dictionary. What are a few features or relationships you might be interested in checking out?

Answer: relationship between age and class. relationship between blood pressure and class. relationship between apettite and class.

---

## Step 3: Explore the data.

### 3. How much of the data is missing from each column?

In [3]:
df.isnull().sum()

age        9
bp        12
sg        47
al        46
su        49
rbc      152
pc        65
pcc        4
ba         4
bgr       44
bu        19
sc        17
sod       87
pot       88
hemo      52
pcv       71
wbcc     106
rbcc     131
htn        2
dm         2
cad        2
appet      1
pe         1
ane        1
class      0
dtype: int64

Only class has no missing values.

### 4. Suppose that I dropped every row that contained at least one missing value. (In the context of analysis with missing data, we call this a "complete case analysis," because we keep only the complete cases!) How many rows would remain in our dataframe? What are at least two downsides to doing this?

> There's a good visual on slide 15 of [this deck](https://liberalarts.utexas.edu/prc/_files/cs/Missing-Data.pdf) that shows what a complete case analysis looks like if you're interested.

Answer: There would be 158 rows. One downside to doing this is that we would lose alot of data. Another downside is the rows that are not complete might not be uncomplete at random which will skew our data when we drop them.

### 5. Thinking critically about how our data were gathered, it's likely that these records were gathered by doctors and nurses. Brainstorm three potential areas (in addition to the missing data we've already discussed) where this data might be inaccurate or imprecise.

Answer: the data could be entered incorrectly because they have to rush. They could take more detailed measurements on cases they thought were serious. The measurements might not be precise.

---

## Step 4: Model the data.

### 6. Suppose that I want to construct a model where no person who has CKD will ever be told that they do not have CKD. What (very simple, no machine learning needed) model can I create that will never tell a person with CKD that they do not have CKD?

> Hint: Don't think about `statsmodels` or `scikit-learn` here.

Answer: the model is that all patients are assumed to have CKD.

### 7. In problem 6, what common classification metric did we optimize for? Did we minimize false positives or negatives?

Answer: False negatives were minimized. Sensitivity will be optimized. The model will have a sensitivity of 1. 

### 8. Thinking ethically, what is at least one disadvantage to the model you described in problem 6?

Answer: It is not ethical to tell everyone they have CKD. Then even people who don't need to get further doctors appointments and tests will have to get them. And that will be expensive for them.

### 9. Suppose that I want to construct a model where a person who does not have CKD will ever be told that they do have CKD. What (very simple, no machine learning needed) model can I create that will accomplish this?

Answer: the model is all patients are told they don't have CKD.

### 10. In problem 9, what common classification metric did we optimize for? Did we minimize false positives or negatives?

Answer: False positives were minimized. Specificity was maximized.

### 11. Thinking ethically, what is at least one disadvantage to the model you described in problem 9?

Answer: People who have CKD will not get treatment. They will not even know they have the very serious problem.

### 12. Construct a logistic regression model in `sklearn` predicting class from the other variables. You may scale, select/drop, and engineer features as you wish - build a good model! Make sure, however, that you include at least one categorical/dummy feature and at least one quantitative feature.

> Hint: Remember to do a train/test split!

In [4]:
df =df.dropna()

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 158 entries, 3 to 399
Data columns (total 25 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   age     158 non-null    float64
 1   bp      158 non-null    float64
 2   sg      158 non-null    float64
 3   al      158 non-null    float64
 4   su      158 non-null    float64
 5   rbc     158 non-null    object 
 6   pc      158 non-null    object 
 7   pcc     158 non-null    object 
 8   ba      158 non-null    object 
 9   bgr     158 non-null    float64
 10  bu      158 non-null    float64
 11  sc      158 non-null    float64
 12  sod     158 non-null    float64
 13  pot     158 non-null    float64
 14  hemo    158 non-null    float64
 15  pcv     158 non-null    float64
 16  wbcc    158 non-null    float64
 17  rbcc    158 non-null    float64
 18  htn     158 non-null    object 
 19  dm      158 non-null    object 
 20  cad     158 non-null    object 
 21  appet   158 non-null    object 
 22  pe

In [6]:
dum1 = pd.get_dummies(df['rbc'], drop_first =True)
dum2 = pd.get_dummies(df['pc'], drop_first =True)
dum3 = pd.get_dummies(df['pcc'], drop_first =True)
dum4 = pd.get_dummies(df['ba'], drop_first =True)
dum5 = pd.get_dummies(df['htn'], drop_first =True)
dum6 = pd.get_dummies(df['dm'], drop_first =True)
dum7 = pd.get_dummies(df['cad'], drop_first =True)
dum8 = pd.get_dummies(df['appet'], drop_first =True)
dum9 = pd.get_dummies(df['pe'], drop_first =True)
dum10 = pd.get_dummies(df['ane'], drop_first =True)
dum11 = pd.get_dummies(df['class'], drop_first =True)

In [7]:
df = df.join(dum5)
df = df.join(dum11)

In [8]:
df.head()

,age,bp,sg,al,su,rbc,pc,pcc,ba,bgr,...,rbcc,htn,dm,cad,appet,pe,ane,class,yes,notckd
3,48.0,70.0,1.005,4.0,0.0,normal,abnormal,present,notpresent,117.0,...,3.9,yes,no,no,poor,yes,yes,ckd,1,0
9,53.0,90.0,1.020,2.0,0.0,abnormal,abnormal,present,notpresent,70.0,...,3.7,yes,yes,no,poor,no,yes,ckd,1,0
11,63.0,70.0,1.010,3.0,0.0,abnormal,abnormal,present,notpresent,380.0,...,3.8,yes,yes,no,poor,yes,no,ckd,1,0
14,68.0,80.0,1.010,3.0,2.0,normal,abnormal,present,present,157.0,...,2.6,yes,yes,yes,poor,yes,no,ckd,1,0
20,61.0,80.0,1.015,2.0,0.0,abnormal,abnormal,notpresent,notpresent,173.0,...,3.2,yes,yes,yes,poor,yes,yes,ckd,1,0


In [21]:
df['class'].value_counts()

notckd    115
ckd        43
Name: class, dtype: int64

In [22]:
df['notckd'].value_counts()

1    115
0     43
Name: notckd, dtype: int64

In [10]:
features =['age','bp','sg','al','su','sod','pot','hemo','pcv','yes']
X = df[features]
y = df['notckd']

In [11]:
sc = StandardScaler()
Xs = sc.fit_transform(X) #scale features

In [12]:
X_train, X_test, y_train, y_test = train_test_split(Xs, y, random_state = 42)

In [13]:
logreg1 = LogisticRegression(solver="liblinear")
logreg1.fit(X_train, y_train)

LogisticRegression(solver='liblinear')

In [14]:
preda = logreg1.predict(X_train)

In [15]:
pred1 = logreg1.predict(X_test)

In [16]:
accuracy_score(y_train, preda)

1.0

In [17]:
accuracy_score(y_test, pred1)

1.0

---

## Step 5: Evaluate the model.

### 13. Based on your logistic regression model constructed in problem 12, interpret the coefficient of one of your quantitative features.

In [18]:
logreg1.coef_

array([[-0.2352434 , -0.19717614,  0.72864563, -1.56511113, -0.59098746,
         0.58004469, -0.03520473,  0.87630336,  0.86760322, -0.75524469]])

The age coefficient is the first one. A one standard deviation increase in age means the log odds of not having ckd decreases by -0.23 all other things being constant.

### 14. Based on your logistic regression model constructed in problem 12, interpret the coefficient of one of your categorical/dummy features.

The htn coefficient is the last one. A one standard deviation increase in having hypertension means the log odds of not having ckd decreases by -0.75 all other things being constant. That interpretation does not really make sense for dummy variables. How can a binary variable increase by one standard deviation.

### 15. Despite being a relatively simple model, logistic regression is very widely used in the real world. Why do you think that's the case? Name at least two advantages to using logistic regression as a modeling technique.

Answer: Logistic regression is easier to interpret then black box models. And you can use regularization to avoid overfitting.

### 16. Does it make sense to generate a confusion matrix on our training data or our test data? Why? Generate it on the proper data.

> Hint: Once you've generated your predicted $y$ values and you have your observed $y$ values, then it will be easy to [generate a confusion matrix using sklearn](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html).

We want to generate it on the test data to see how the model is generalizing to new data.

### 17. In this hospital case, we want to predict CKD. Do we want to optimize for sensitivity, specificity, or something else? Why? (If you don't think there's one clear answer, that's okay! There rarely is. Be sure to defend your conclusion!)

Answer: This is a hard one. I think the worst outcome is a false negative where someone has CKD and we dont find it. Therefore sensitivity is what we want to maximize. The reason for this is if someone gets a false positive they may have to pay for more expensive tests but it won't hurt their health. But if someone gets a false negative their condition goes untreated.

### 18 (BONUS). Write a function that will create an ROC curve for you, then plot the ROC curve.

Here's a strategy you might consider:
1. In order to even begin, you'll need some fit model. Use your logistic regression model from problem 12.
2. We want to look at all values of your "threshold" - that is, anything where .predict() gives you above your threshold falls in the "positive class," and anything that is below your threshold falls in the "negative class." Start the threshold at 0.
3. At this value of your threshold, calculate the sensitivity and specificity. Store these values.
4. Increment your threshold by some "step." Maybe set your step to be 0.01, or even smaller.
5. At this value of your threshold, calculate the sensitivity and specificity. Store these values.
6. Repeat steps 3 and 4 until you get to the threshold of 1.
7. Plot the values of sensitivity and 1 - specificity.

### 19. Suppose you're speaking with the biostatistics lead at Mayo Clinic, who asks you "Why are unbalanced classes generally a problem? Are they a problem in this particular CKD analysis?" How would you respond?

Answer: Unbalanced classes are an issue because the model could have a high accuracy by always predicting the majority class. It is an issue there because the classes are not balanced.

### 20. Suppose you're speaking with a doctor at Mayo Clinic who, despite being very smart, doesn't know much about data science or statistics. How would you explain why unbalanced classes are generally a problem to this doctor?

Answer: Unbalanced classes are when one of our binary classes are inequal in terms of size. The issue here is that the model could be very accurate by just predicting everything as the majority class. However then we are not getting an information out of the model.

### 21. Let's create very unbalanced classes just for the sake of this example! Generate very unbalanced classes by [bootstrapping](http://stattrek.com/statistics/dictionary.aspx?definition=sampling_with_replacement) (a.k.a. random sampling with replacement) the majority class.

1. The majority class are those individuals with CKD.
2. Generate a random sample of size 200,000 of individuals who have CKD **with replacement**. (Consider setting a random seed for this part!)
3. Create a new dataframe with the original data plus this random sample of data.
4. Now we should have a dataset with around 200,000 observations, of which only about 0.00075% are non-CKD individuals.

In [23]:
xsamp = df[df['class'] == 'ckd'].sample(n= 200000, replace =True, random_state =42)

In [25]:
new_df =pd.concat([df, xsamp])

In [27]:
new_df['class'].value_counts()

ckd       200043
notckd       115
Name: class, dtype: int64

### 22. Build a logistic regression model on the unbalanced class data and evaluate its performance using whatever method(s) you see fit. How would you describe the impact of unbalanced classes on logistic regression as a classifier?
> Be sure to look at how well it performs on non-CKD data.

In [28]:
features =['age','bp','sg','al','su','sod','pot','hemo','pcv','yes']
X = new_df[features]
y = new_df['notckd']

In [29]:
sc = StandardScaler()
Xs = sc.fit_transform(X)

In [30]:
X_train, X_test, y_train, y_test = train_test_split(Xs, y, random_state = 42)

In [31]:
logreg1 = LogisticRegression(solver="liblinear")
logreg1.fit(X_train, y_train)

LogisticRegression(solver='liblinear')

In [32]:
preda = logreg1.predict(X_train)

In [33]:
pred1 = logreg1.predict(X_test)

In [35]:
accuracy_score(y_train, preda)

1.0

In [36]:
accuracy_score(y_test, pred1)

1.0

In [40]:
confusion_matrix(y_train, preda)

array([[150038,      0],
       [     0,     80]], dtype=int64)

In [41]:
confusion_matrix(y_test, pred1)

array([[50005,     0],
       [    0,    35]], dtype=int64)

The model predicted that everyone in the data has ckd. When the data is imbalanced enough logistic regression is useless.

---

## Step 6: Answer the problem.

At this step, you would generally answer the problem! In this situation, you would likely present your model to doctors or administrators at the hospital and show how your model results in reduced false positives/false negatives. Next steps would be to find a way to roll this model and its conclusions out across the hospital so that the outcomes of patients with CKD (and without CKD!) can be improved!